In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
citeseer_dataset = Planetoid(root='', name='Citeseer')
data = citeseer_dataset[0].to(device)
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [15]:
model = GCN(data.x.shape[1], citeseer_dataset.num_classes, [64]).to(device)

In [16]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 2.764373540878296, Val Loss - 4.158348083496094, Val Accuracy - 0.351
Epoch 20, Train Loss - 0.18402592837810516, Val Loss - 3.855626106262207, Val Accuracy - 0.628
Epoch 40, Train Loss - 0.008632439188659191, Val Loss - 4.55294942855835, Val Accuracy - 0.608
Epoch 60, Train Loss - 0.12433706969022751, Val Loss - 4.521960735321045, Val Accuracy - 0.621
Epoch 80, Train Loss - 0.053415264934301376, Val Loss - 4.821153163909912, Val Accuracy - 0.614
Epoch 100, Train Loss - 0.021264292299747467, Val Loss - 5.989776134490967, Val Accuracy - 0.57
Epoch 120, Train Loss - 0.008047237992286682, Val Loss - 6.646504878997803, Val Accuracy - 0.563
Epoch 140, Train Loss - 0.09386077523231506, Val Loss - 6.1540398597717285, Val Accuracy - 0.605
Epoch 160, Train Loss - 0.0053002662025392056, Val Loss - 5.723100662231445, Val Accuracy - 0.608
Epoch 180, Train Loss - 0.024926424026489258, Val Loss - 5.550207614898682, Val Accuracy - 0.596
Epoch 200, Train Loss - 0.0054176240228116

In [17]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.6
Initial Loss: 5.303683280944824


In [7]:
amts = defaultdict(int)
# Run 10 attacks for perturbation of 0.2
for i in range(10):
    attacker = Metattack(data, device=device)
    attacker.reset_parameters(seed=i)
    attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)
    attacker.reset()
    attacker.attack(0.2)

    for edge in attacker._added_edges.keys():
        amts[edge] += 1


Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

In [8]:
amts

defaultdict(int,
            {(107, 806): 10,
             (31, 3136): 10,
             (119, 2787): 10,
             (53, 2813): 10,
             (113, 1372): 10,
             (118, 1841): 10,
             (42, 2334): 10,
             (48, 1626): 10,
             (31, 939): 10,
             (31, 788): 10,
             (42, 67): 10,
             (110, 659): 10,
             (42, 2605): 10,
             (42, 2438): 10,
             (42, 196): 10,
             (30, 3173): 10,
             (25, 3170): 10,
             (10, 3315): 10,
             (42, 178): 10,
             (42, 3270): 10,
             (42, 973): 10,
             (42, 89): 10,
             (48, 3191): 10,
             (53, 3224): 10,
             (115, 1787): 10,
             (42, 467): 10,
             (42, 2624): 10,
             (113, 3058): 10,
             (31, 240): 10,
             (31, 3079): 10,
             (42, 2158): 10,
             (31, 2917): 10,
             (42, 105): 10,
             (42, 3094): 10,
    

In [9]:
sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=True)
sorted_keys = [key for key, value in sorted_list]

In [10]:
edges_to_add = sorted_keys

In [11]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [12]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.15
budget = int(ptb_rate * initial_edge_count)

In [13]:
print(len(edges_to_add))
print(budget)

446
341


In [14]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [13]:
splits = [0, 0.5, 0.7, 0.9]
split_dic = defaultdict(list)

In [14]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.0750
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/341 [00:00<?, ?it/s]

Final Accuracy: 0.5370
Accuracy Drop: 0.0220
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0375
Phase 1: Added 170 edges out of requested 170.


Peturbing graph...:   0%|          | 0/177 [00:00<?, ?it/s]

Final Accuracy: 0.5330
Accuracy Drop: 0.0260
0.7

--- Running split: 0.7 ---
Second phase perturbation rate: 0.0225
Phase 1: Added 238 edges out of requested 238.


Peturbing graph...:   0%|          | 0/107 [00:00<?, ?it/s]

Final Accuracy: 0.5360
Accuracy Drop: 0.0230
0.9

--- Running split: 0.9 ---
Second phase perturbation rate: 0.0075
Phase 1: Added 306 edges out of requested 306.


Peturbing graph...:   0%|          | 0/36 [00:00<?, ?it/s]

Final Accuracy: 0.5460
Accuracy Drop: 0.0130


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
